In [1]:
%%writefile reliance_deploy.py
import pandas as pd
import yfinance as yf
import streamlit as st
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt
import plotly.express as pe
import plotly.graph_objects as go
from datetime import date
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

st.markdown(
    """
    <style>
    body {
        background-color: Gray; /* Light gray background color */
    }
    .stApp {
        background-color: Gray;
    }
    .sidebar .sidebar-content {
        background-color: Violet; /* Slightly darker color for the sidebar */
    }
    </style>
    """,
    unsafe_allow_html=True
)
st.title("Stock Price Forecasting of Reliance")

def count_weekdays(start_date, end_date):
    total_days = 0
    current_date = start_date
    while current_date <= end_date:
        if current_date.weekday() < 5: 
            total_days += 1
        current_date += timedelta(days=1)
    return total_days
    
with st.sidebar:
    st.header("Days Calculator")
    
    user_date = st.text_input("Enter a future date (YYYY-MM-DD):")
    if st.button("Calculate"):
        if user_date:
            try:
                today = datetime.now().date()
                conv_date = datetime.strptime(user_date, '%Y-%m-%d').date()
                diff = count_weekdays(today, conv_date)
                st.sidebar.write(f'Forecast Step for {user_date} is {diff} days.')
            except ValueError:
                st.sidebar.error("Please enter a valid date in the format YYYY-MM-DD.")
        else:
            st.sidebar.error("Please enter a future date.") 

end = date.today().strftime("%Y-%m-%d")
df = yf.download('RELIANCE.NS', start='2010-01-01', end=end)
data = df[['Close']]

with st.form(key='date_form'):
    st.header("Forecast Step:")
    forecast_stepss = st.number_input("Specify the number of days:", min_value=1, value=10, step=1)
    st.write("Days:", forecast_stepss)
    submit_button = st.form_submit_button(label="Submit")

if submit_button:

    try:
        scaler = StandardScaler()
        data_scaled = scaler.fit_transform(data)

        model = SARIMAX(data_scaled, order=(1,1,1), seasonal_order=(1,1,1,12))
        model_fit = model.fit(disp=False)

        forecast = model_fit.get_forecast(steps=forecast_stepss)
        forecast_index = pd.date_range(start=data.index[-1] + pd.DateOffset(days=1), periods=forecast_stepss, freq='B')
        forecast_values = scaler.inverse_transform(forecast.predicted_mean.reshape(-1, 1))

        forecast_df = pd.DataFrame(data=forecast_values, index=forecast_index, columns=['Forecast'])
        st.write("Here is your", forecast_stepss, "days prediction:\n\n")
        st.write(forecast_df)
        

        fig, ax = plt.subplots(figsize=(14, 7))
        ax.plot(forecast_df.index, forecast_df['Forecast'], color='orange', linestyle='--', marker='*')
        ax.set_xlabel('Date')
        ax.set_ylabel('Forecast')
        ax.set_title('Forecast Data')
        ax.grid(True)
        st.pyplot(fig)  
                
        fig = pe.line(forecast_df, x=forecast_df.index, y='Forecast', title='Forecast Data')
        st.plotly_chart(fig) 

    except Exception as e:
        st.error(f"Error during forecasting: {e}")

Writing reliance_deploy.py
